In [2]:
!pip install kaggle

In [ ]:
#configuring path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Configured")

Configured


In [ ]:
#verify configuration
!ls -l ~/.kaggle/kaggle.json
  

-rw-------@ 1 subhashbandaraekanayake  staff  73 Nov 17 08:42 /Users/subhashbandaraekanayake/.kaggle/kaggle.json


In [6]:
!kaggle datasets download -d kazanova/sentiment140

100%|██████████████████████████████████████| 80.9M/80.9M [00:31<00:00, 2.80MB/s]
100%|██████████████████████████████████████| 80.9M/80.9M [00:31<00:00, 2.65MB/s]


In [7]:
#extracting compressed dataset
from zipfile import ZipFile
dataset = '/Users/subhashbandaraekanayake/Desktop/MyDSJ/Projects/NLP projects/Twitter sentiment analysis/twitter-sentiment-analysis-NLP/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
    zip.extractall()
    print("dataset is extracted")

dataset is extracted
